In [0]:
%sql
-- COMMAND ----------
-- Create Catalog
CREATE CATALOG telecom_catalog_assign;

-- COMMAND ----------
-- Create Schema inside the catalog
CREATE SCHEMA telecom_catalog_assign.landing_zone;

-- COMMAND ----------
-- Create Volume inside the schema
CREATE VOLUME telecom_catalog_assign.landing_zone.landing_vol;

In [0]:
# COMMAND ----------
# Create folders inside the volume using dbutils
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

#2. Filesystem operations

In [0]:
# COMMAND ----------
# Define dataset strings
customer_csv = """101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID"""

usage_tsv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0"""

tower_logs_region1 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12"""

tower_logs_region2 = """event_id|customer_id|tower_id|signal_strength|timestamp
6001|102|TWR02|-70|2025-01-11 09:15:33
6002|103|TWR03|-85|2025-01-11 09:45:10"""

In [0]:
# COMMAND ----------
# Copy datasets into Volume folders
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", customer_csv, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv", usage_tsv, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.log", tower_logs_region1, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region2.log", tower_logs_region2, True)

In [0]:
# COMMAND ----------
# Validate files in customer folder
display(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/"))

# Validate files in usage folder
display(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/"))

# Validate files in tower/region1 folder
display(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/"))

# Validate files in tower/region2 folder
display(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/"))

# 3. Directory Read Use Cases

In [0]:
# COMMAND ----------
tower_glob_df = spark.read.option("delimiter", "|").option("header", True) \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*/*.log")

display(tower_glob_df)

In [0]:
# COMMAND ----------
tower_multi_df = spark.read.option("delimiter", "|").option("header", True) \
    .csv([
        "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.log",
        "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region2.log"
    ])

display(tower_multi_df)

In [0]:
# COMMAND ----------
tower_recursive_df = spark.read.option("delimiter", "|").option("header", True) \
    .option("recursiveFileLookup", "true") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

display(tower_recursive_df)

# 4. Schema Inference, Header, and Separator

In [0]:
customer_df = spark.read.option("header", "false").option("inferSchema", "false") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

usage_df = spark.read.option("header", "false").option("inferSchema", "false") \
    .option("delimiter", "\t") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")

In [0]:
print(customer_df)
print(usage_df)

In [0]:
customer_df = spark.read.option("header", "true").option("inferSchema", "true") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

usage_df = spark.read.option("header", "true").option("inferSchema", "true") \
    .option("delimiter", "\t") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")

In [0]:
print(customer_df)
print(usage_df)

# 5. Column Renaming Usecases

In [0]:
# COMMAND ----------
# Read raw customer data (no header, all strings)
customer_df_raw = spark.read.option("header", "false").csv(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv"
)

# Apply column names using toDF
customer_df = customer_df_raw.toDF("customer_id", "name", "age", "city", "plan_type")

display(customer_df)

In [0]:
# COMMAND ----------
# Define schema using string
usage_schema = "customer_id INT, voice_mins INT, data_mb INT, sms_count INT"

# Read usage data with schema
usage_df = spark.read.option("header", "true").option("delimiter", "\t") \
    .schema(usage_schema) \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")

display(usage_df)